# 006: Operation Graphs - Academic Claim Validation

Building on ReAct patterns from tutorial 005, this demonstrates sequential coordination using Operation Graphs to orchestrate ReaderTool workflows for academic claim validation.

## What You'll Learn

1. **Sequential Coordination**: Building context step-by-step through dependent operations
2. **ReaderTool Integration**: Document chunking and progressive reading strategies  
3. **Structured Extraction**: Using Pydantic models for reliable claim extraction
4. **Operation Dependencies**: How operations build on previous results

## Use Case: Validating a Theoretical Framework Paper

We'll validate claims in an academic paper about capability-based security by:
- Reading document chunks progressively with ReaderTool
- Building understanding through sequential analysis
- Extracting verifiable claims with structured output formats
- Demonstrating how each operation builds on the previous one's results

In [1]:
# Setup and imports
from typing import Literal
from pathlib import Path
from pydantic import BaseModel, Field

from lionagi import Branch, Session, Builder, types, iModel
from lionagi.tools.types import ReaderTool

# Target document - complex theoretical framework
here = Path().cwd()
document_path = here / "data" / "006_lion_proof_ch2.md"

print("✅ Environment setup complete")
print(f"📄 Target: {document_path.name}")
print("🎯 Goal: Validate academic claims using coordinated ReAct workflows")

✅ Environment setup complete
📄 Target: 006_lion_proof_ch2.md
🎯 Goal: Validate academic claims using coordinated ReAct workflows


In [2]:
# Data models for structured responses
class Claim(BaseModel):
    claim: str
    type: Literal["citation", "performance", "technical", "other"]
    location: str = Field(..., description="Section/paragraph reference")
    verifiability: Literal["high", "medium", "low"]
    search_strategy: str = Field(..., description="How to verify this claim")


class ClaimExtraction(BaseModel):
    claims: list[Claim]


print("✅ Data models defined")

✅ Data models defined


## Pattern 1: Sequential Document Analysis

Build understanding step-by-step: Open → Analyze → Extract claims

In [3]:
async def sequential_analysis():
    """Sequential workflow: open → analyze structure → extract claims."""

    # Create branch with ReaderTool
    branch = Branch(
        tools=[ReaderTool], chat_model=iModel(model="openai/gpt-5-mini")
    )
    session = Session(default_branch=branch)
    builder = Builder()

    # Step 1: Open and understand document
    doc_reader = builder.add_operation(
        "ReAct",
        node_id="open_document",
        instruct=types.Instruct(
            instruction="Use ReaderTool to open and analyze the theoretical framework document. Understand its structure and identify sections containing verifiable claims.",
            context={"document_path": str(document_path)},
        ),
        tools=["reader_tool"],
        max_extensions=2,
        verbose=True,
        verbose_length=1000,
    )

    # Step 2: Progressive content analysis
    content_analyzer = builder.add_operation(
        "ReAct",
        node_id="analyze_content",
        depends_on=[doc_reader],
        instruct=types.Instruct(
            instruction="Read through key sections to identify citations, technical claims, and performance metrics that can be verified."
        ),
        response_format=types.Outline,
        tools=["reader_tool"],
        max_extensions=3,
        verbose=True,
        verbose_length=1000,
        inherit_context=True,
    )

    # Step 3: Extract specific claims
    claim_extractor = builder.add_operation(
        "ReAct",
        node_id="extract_claims",
        depends_on=[content_analyzer],
        instruct=types.Instruct(
            instruction="Extract 5-7 specific, verifiable claims. Prioritize citations, performance metrics, and technical assertions."
        ),
        response_format=ClaimExtraction,
        tools=["reader_tool"],
        max_extensions=3,
        verbose=True,
        verbose_length=1000,
        inherit_context=True,
    )

    # Execute workflow
    graph = builder.get_graph()
    print("🔗 Executing sequential analysis...")

    result = await session.flow(graph, parallel=False, verbose=True)

    return result


# Execute sequential analysis
result = await sequential_analysis()

🔗 Executing sequential analysis...
Pre-allocated 2 branches
Executing operation: e9e8faff


2025-10-15 07:50:15,795 - INFO - detected formats: [<InputFormat.MD: 'md'>]
2025-10-15 07:50:15,796 - INFO - Going to convert document batch...
2025-10-15 07:50:15,796 - INFO - Initializing pipeline for SimplePipeline with options hash 995a146ad601044538e6a923bea22f4e
2025-10-15 07:50:15,804 - INFO - Loading plugin 'docling_defaults'
2025-10-15 07:50:15,805 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-10-15 07:50:15,806 - INFO - Processing document 006_lion_proof_ch2.md
2025-10-15 07:50:17,017 - INFO - Finished converting document 006_lion_proof_ch2.md in 1.22 sec.


### ReAct Round No.1 Analysis:

```yaml
analysis: I will first open the specified markdown document so I can read its contents. After opening, I will read the document in sequential slices (start-to-end or by sections) to understand its structure (headings, subsections, theorems/claims, proofs, citations). For each major section and subsection I will identify and list statements that are verifiable (e.g., explicit theorems, lemmas, propositions, empirical claims with citations, or factual assertions referencing external results). I will also note where claims lack citations or appear to be definitions/conventions (which are not verifiable in the same way). This stepwise approach reduces I/O size and lets me confirm findings incrementally.
planned_actions:
  - action_type: reader_tool
    description: Open the file /Users/lion/projects/lionagi/cookbooks/data/006_lion_proof_ch2.md so its text is available for subsequent partial reads. Rationale: must load the document before reading and analyzing its structure and veri...

[Truncated output]
```

### ReAct Round No.2 Analysis:

```yaml
analysis: I will read the opened document in sequential slices and extract its structure (headings, subsections, theorems/lemmas/definitions, proofs, citations) and identify candidate verifiable claims (theorems, propositions, lemmas, empirical/factual assertions that can be checked against literature or computation). To start, I'll read the initial portion of the document (characters 0–10000) to detect top-level headings and the first set of claims. After that chunk is analyzed, I'll request further reads until the full document is covered or until all verifiable claims are collected.
planned_actions:
  - action_type: reader_tool
    description: Read the first 10,000 characters of the opened document DOC_6453347028123389634 to identify headings and initial verifiable claims. Rationale: large document (53006 chars) — chunked reads reduce I/O and let me extract structure incrementally.
extension_needed: True
milestone: Read initial chunk (0-10000) and extract headings and candi...

[Truncated output]
```

### ReAct Round No.3 Analysis:

```yaml
analysis: |
  I opened and read the first 10,000 characters of the document DOC_6453347028123389634. In this chunk I located the chapter title, author/date, Introduction (2.1), System Model and Formal Definitions (2.2), and the beginning of Theorem 2.1 (2.3) with its proof structure. Key structural elements found so far:
  
  - Top-level headings: 2.1 Introduction, 2.2 System Model and Formal Definitions, 2.3 Theorem 2.1
  - Subsections: 2.1.1 Motivation, 2.1.2 Contribution Overview, 2.2.1 Lion Ecosystem Architecture, 2.2.2 Formal System Definition, 2.3.1 Theorem Statement, 2.3.2 Proof Structure
  - Formal definitions: Definition 2.1 (Lion Capability System as 7-tuple), Definition 2.2 (Cross-Component Capability 5-tuple), Definition 2.3 (Capability Authority), Definition 2.4 (Component Composition), Definition 2.5 (Security Properties)
  - Theorems/lemmas encountered: Theorem 2.1 (Cross-Component Capability Flow), Lemma 2.1.1 (WebAssembly Isolation Preserves Capability Referenc...

[Truncated output]
```

...

### ReAct Final Answer:

```yaml
topic: Capability-Based Security Framework: Citations, Technical Claims, and Performance Metrics
items:
  - heading: Foundational Citations
    summary: Includes seminal works in capability security and OS protection like Dennis & Van Horn (1966), Saltzer & Schroeder (1975), Lampson (1974), and Hardy (1988), providing the theoretical basis.
  - heading: Modern Capability Systems References
    summary: Citations of recent advances and implementations including Miller (2006), Shapiro et al. (1999), and Mettler & Wagner (2008, 2010), contextualizing the framework within current research.
  - heading: Formal Verification Literature
    summary: Key formal methods works are cited, such as Klein et al. (2009) on seL4, and Lean/TLA+ mechanizations supporting the theorems proved in the document.
  - heading: Technical Claims in Implementation
    summary: Claims on cryptographic integrity (HMAC-SHA256 providing unforgeability and authentication), type-level enforcement of least privil...

[Truncated output]
```

Completed operation: 8642dd92
Operation 9a6a2769 waiting for 8642dd92
Operation 9a6a2769 inherited context from 8642dd92
Executing operation: 9a6a2769


### ReAct Round No.1 Analysis:

```yaml
analysis: |
  Based on the previously read content of the theoretical framework document, I identify the following 6 specific, verifiable claims prioritized by citations, technical assertions, and performance metrics:
  
  1. Theorem 2.1 states that the authority of capability tokens is preserved across component boundaries and tokens remain cryptographically unforgeable during inter-component communication. This is supported by both formal proofs and a TLA+ mechanized model (Appendix A.2).
  
  2. The Rust implementation enforces capability integrity and least privilege through phantom types, type-level constraints, and runtime WebAssembly isolation that prevents forgery and enforces authority attenuation safely.
  
  3. Cryptographic integrity of capabilities is provided by HMAC-SHA256 signatures with 128-bit security, ensuring only Lion core can generate valid capabilities. The implementation includes timestamped signatures and validation code verifiable via cryptographic au...

[Truncated output]
```

### ReAct Final Answer:

```yaml
claims:
  - claim: Theorem 2.1 states that capability tokens preserve their authority and remain cryptographically unforgeable across component boundaries during inter-component communication, as formalized and mechanized in TLA+ (Appendix A.2).
    type: citation
    location: Section 2.3 Theorem 2.1 and Appendix A.2
    verifiability: high
    search_strategy: Review the formal proof and TLA+ mechanization in Appendix A.2; verify correspondence between the model and the theorem statement, and validate assumptions about cryptographic unforgeability.
  - claim: The Rust implementation enforces capability integrity and least privilege at compile time using phantom types and type-level constraints, complemented by runtime WebAssembly isolation which prevents forgery of capabilities.
    type: technical
    location: Section 2.10.4 and 2.11.1 Rust Implementation Architecture
    verifiability: high
    search_strategy: Examine the Rust code snippets provided, verify the use of pha...

[Truncated output]
```

Completed operation: 9a6a2769


In [4]:
from IPython.display import display, Markdown

# Display results
for node_id, data in result["operation_results"].items():
    if isinstance(data, types.Outline):
        md_content = f"""
## 📄 Document Structure ({node_id})

**Topic:** {data.topic}

### Key Sections:
"""
        for item in data.items[:3]:  # Show first 3
            md_content += f"- **{item.heading}**: {item.summary}\n"

        display(Markdown(md_content))

    elif isinstance(data, ClaimExtraction):
        md_content = f"""
## 📑 Extracted Claims ({node_id})

Found **{len(data.claims)}** verifiable claims:

"""
        for i, claim in enumerate(data.claims, 1):
            md_content += f"""
### {i}. [{claim.type.upper()}] {claim.claim}

- **Location:** {claim.location}  
- **Verifiability:** {claim.verifiability}
- **Search Strategy:** {claim.search_strategy}

"""
        display(Markdown(md_content))

display(Markdown("## ✅ Sequential analysis completed"))


## 📄 Document Structure (8642dd92-099d-494b-8c0d-280943d8096f)

**Topic:** Capability-Based Security Framework: Citations, Technical Claims, and Performance Metrics

### Key Sections:
- **Foundational Citations**: Includes seminal works in capability security and OS protection like Dennis & Van Horn (1966), Saltzer & Schroeder (1975), Lampson (1974), and Hardy (1988), providing the theoretical basis.
- **Modern Capability Systems References**: Citations of recent advances and implementations including Miller (2006), Shapiro et al. (1999), and Mettler & Wagner (2008, 2010), contextualizing the framework within current research.
- **Formal Verification Literature**: Key formal methods works are cited, such as Klein et al. (2009) on seL4, and Lean/TLA+ mechanizations supporting the theorems proved in the document.



## 📑 Extracted Claims (9a6a2769-c760-4a4d-bd2d-f60fdbb74d47)

Found **6** verifiable claims:


### 1. [CITATION] Theorem 2.1 states that capability tokens preserve their authority and remain cryptographically unforgeable across component boundaries during inter-component communication, as formalized and mechanized in TLA+ (Appendix A.2).

- **Location:** Section 2.3 Theorem 2.1 and Appendix A.2  
- **Verifiability:** high
- **Search Strategy:** Review the formal proof and TLA+ mechanization in Appendix A.2; verify correspondence between the model and the theorem statement, and validate assumptions about cryptographic unforgeability.


### 2. [TECHNICAL] The Rust implementation enforces capability integrity and least privilege at compile time using phantom types and type-level constraints, complemented by runtime WebAssembly isolation which prevents forgery of capabilities.

- **Location:** Section 2.10.4 and 2.11.1 Rust Implementation Architecture  
- **Verifiability:** high
- **Search Strategy:** Examine the Rust code snippets provided, verify the use of phantom types and type constraints, and audit the WebAssembly isolation design and implementation within the lion_core project.


### 3. [TECHNICAL] Capability cryptographic integrity is ensured via HMAC-SHA256 signatures providing 128-bit security, with only the Lion core able to generate valid signatures, including timestamped verification to prevent forgery.

- **Location:** Section 2.11.2 Cryptographic Implementation Details  
- **Verifiability:** high
- **Search Strategy:** Audit the HMAC-SHA256 implementation and usage in the Rust code, review cryptographic key management practices, and confirm timestamp usage for replay protection.


### 4. [PERFORMANCE] Capability creation, authority verification, and cross-component capability transfer operate with constant-time (O(1)) complexity and microsecond-level latencies, e.g., capability creation latency of 2.3 microseconds and authority verification latency of 0.8 microseconds.

- **Location:** Section 2.11.3 Performance Characteristics (performance table)  
- **Verifiability:** high
- **Search Strategy:** Reproduce performance benchmarks using the Lion implementation environment, confirm measurement methodology aligns with stated complexity and latency values.


### 5. [CITATION] Theorem 2.2 proves that secure components composed via compatible interfaces preserve their security properties in the composite system, supported by a mechanized proof in the Lean theorem prover.

- **Location:** Section 2.4 Theorem 2.2 and Lean mechanization reference  
- **Verifiability:** high
- **Search Strategy:** Review the formal proof in the document and inspect the Lean mechanized proof files; verify the assumptions on component interfaces and the preservation of security predicates.


### 6. [TECHNICAL] The capability system architecture prevents confused deputy attacks by eliminating ambient authority and requiring explicit capability transfer before privileged actions, formalized in Theorem 2.3.

- **Location:** Section 2.5 Theorem 2.3  
- **Verifiability:** medium
- **Search Strategy:** Examine the formal statement and proof of Theorem 2.3; cross-reference related literature on confused deputy prevention to confirm claim’s theoretical validity.



## ✅ Sequential analysis completed